In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install wrapt_timeout_decorator
    

    100% |████████████████████████████████| 1.4MB 11.3MB/s ta 0:00:01
  Found existing installation: pip 18.1
    Uninstalling pip-18.1:
      Successfully uninstalled pip-18.1


In [2]:
import time
from wrapt_timeout_decorator import *

@timeout(5)
def mytest(message):
    print(message)
    for i in range(1,10):
        time.sleep(1)
        print('{} seconds have passed'.format(i))

if __name__ == '__main__':
    mytest('starting')

starting
1 seconds have passed
2 seconds have passed
3 seconds have passed
4 seconds have passed


TimeoutError: Function mytest timed out after 5 seconds

In [5]:
import time
from wrapt_timeout_decorator import *

# caveats when using signals - the TimeoutError raised by the signal may be catched
# inside the decorated function.
# So You might use Your own Exception, derived from the base Exception Class.
# In Python-3.7.1 stdlib there are over 300 pieces of code that will catch your timeout
# if you were to base an exception on Exception. If you base your exception on BaseException,
# there are still 231 places that can potentially catch your exception.
# You should use use_signals=False if You want to make sure that the timeout is handled correctly !
# therefore the default value for use_signals = False on this decorator !

@timeout(5, use_signals=True)
def mytest(message):
    try:
        print(message)
        for i in range(1,10):
            time.sleep(1)
            print('{} seconds have passed - lets assume we read a big file here'.format(i))
    # TimeoutError is a Subclass of OSError - therefore it is catched here !
    except OSError:
        for i in range(1,10):
            time.sleep(1)
            print('Whats going on here ? - Ooops the Timeout Exception is catched by the OSError ! {}'.format(i))
    except Exception:
        # even worse !
        pass
    except:
        # the worst - and exists more then 300x in actual Python 3.7 stdlib Code !
        # so You never really can rely that You catch the TimeoutError when using Signals !
        pass


if __name__ == '__main__':
    try:
        mytest('starting')
        print('no Timeout Occured')
    except TimeoutError():
        # this will never be printed because the decorated function catches implicitly the TimeoutError !
        print('Timeout Occured')


starting
1 seconds have passed - lets assume we read a big file here
2 seconds have passed - lets assume we read a big file here
3 seconds have passed - lets assume we read a big file here
4 seconds have passed - lets assume we read a big file here
Whats going on here ? - Ooops the Timeout Exception is catched by the OSError ! 1
Whats going on here ? - Ooops the Timeout Exception is catched by the OSError ! 2
Whats going on here ? - Ooops the Timeout Exception is catched by the OSError ! 3
Whats going on here ? - Ooops the Timeout Exception is catched by the OSError ! 4
Whats going on here ? - Ooops the Timeout Exception is catched by the OSError ! 5
Whats going on here ? - Ooops the Timeout Exception is catched by the OSError ! 6
Whats going on here ? - Ooops the Timeout Exception is catched by the OSError ! 7
Whats going on here ? - Ooops the Timeout Exception is catched by the OSError ! 8
Whats going on here ? - Ooops the Timeout Exception is catched by the OSError ! 9
no Timeout Oc

In [12]:
# nested timeouts
# since there is only ONE ALARM Signal on Unix per process, You need to use use_signals = False for nested timeouts.
# The outmost decorator might use Signals, all nested Decorators needs to use use_signals=False (the default)

# mylib.py
from wrapt_timeout_decorator import *
import time

# this example will work on Windows and Linux
# since the decorated function is not in the __main__ scope but in another module !

@timeout(1, use_signals=True)
def outer():
    inner()

@timeout(30,  use_signals=False)
def inner():
    time.sleep(35)
    print("Should never be printed if you call outer()")
        
if __name__ == '__main__':
    outer()


TimeoutError: Function outer timed out after 1 seconds

Should never be printed if you call outer()


In [3]:
import time
from wrapt_timeout_decorator import *

@timeout(5, use_signals=False)
def mytest(message):
    print(message)
    for i in range(1,10):
        time.sleep(1)
        print('{} seconds have passed'.format(i))

if __name__ == '__main__':
    mytest('starting')

starting
1 seconds have passed
2 seconds have passed
3 seconds have passed
4 seconds have passed


TimeoutError: Function mytest timed out after 5 seconds